In [1]:
library(data.table)

setwd("/project/xinhe/lifan/neuron_stim/mateqtl_100lines_output/final_eqtl_08222023/")

eqtlfs <- list.files(pattern="trans2.5e5_eqtl.txt$")
eqtls <- list()
for(f in eqtlfs) {
  n <- substr(f, 1, nchar(f)-37)
  #eqtls[[n]] <- readRDS(f)$cis$eqtls
  eqtls[[n]] <- fread(f)
}

In [14]:
# Load the reference set of eQTLs extracted from cortex

ref_pairs <- readRDS("../rb_est/ref_pairs.rds")
head(ref_pairs)

gene,SNP,variant_id,gene_id,chr,pos,ref,alt
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>
A1BG,rs3214663,chr19_58346865_GC_G_b38,ENSG00000121410.11,chr19,58346865,GC,G
A1BG-AS1,rs6510137,chr19_58352784_T_C_b38,ENSG00000268895.5,chr19,58352784,T,C
A2M-AS1,rs11611936,chr12_8919566_C_T_b38,ENSG00000245105.2,chr12,8919566,C,T
A2ML1,rs144686314,chr12_8863966_GGC_G_b38,ENSG00000166535.19,chr12,8863966,GGC,G
A4GALT,rs5758896,chr22_42719570_T_C_b38,ENSG00000128274.15,chr22,42719570,T,C
AANAT,rs7224108,chr17_76482816_C_T_b38,ENSG00000129673.9,chr17,76482816,C,T


In [ ]:
snpinfo <- fread("../../snpinfo_100lines.txt")
snpinfo$variant_id <- paste(snpinfo$chr, snpinfo$pos, snpinfo$ref, snpinfo$alt, "b38", sep="_")

In [10]:
### Map to RSID and gene name

cerebellum <- fread("../../GTEx_Analysis_v8_eQTL/Brain_Cerebellum.v8.signif_variant_gene_pairs.txt.gz")
geneinfo <- fread("../../GTEx_Analysis_v8_eQTL/Brain_Cerebellum.v8.egenes.txt.gz")
cere1 <- merge(cerebellum, geneinfo[,1:2], by="gene_id", all.x=T)

cere2 <- merge(cere1, snpinfo[,c("rsid","variant_id")], by="variant_id", all=F)

head(cere2)

variant_id,gene_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,pval_nominal_threshold,min_pval_nominal,pval_beta,gene_name,rsid
<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
chr10_100000235_C_T_b38,ENSG00000107554.16,-9684,102,120,0.287081,4.32963e-06,0.375282,0.0790133,0.000142631,4.24345e-12,2.36930e-08,DNMBP,rs11596870
chr10_100002628_A_C_b38,ENSG00000107554.16,-7291,139,178,0.425837,7.60706e-06,0.347959,0.0753302,0.000142631,4.24345e-12,2.36930e-08,DNMBP,rs11190363
chr10_100004827_A_C_b38,ENSG00000107554.16,-5092,133,167,0.399522,8.98508e-07,0.381298,0.0747442,0.000142631,4.24345e-12,2.36930e-08,DNMBP,rs7902856
chr10_100007241_C_T_b38,ENSG00000107554.16,-2678,133,170,0.406699,1.09631e-06,0.375025,0.0741480,0.000142631,4.24345e-12,2.36930e-08,DNMBP,rs7088596
chr10_100009013_G_A_b38,ENSG00000107554.16,-906,133,170,0.406699,1.09631e-06,0.375025,0.0741480,0.000142631,4.24345e-12,2.36930e-08,DNMBP,rs12267374
chr10_100009635_T_G_b38,ENSG00000095485.16,-258045,135,170,0.406699,4.12072e-05,0.280034,0.0664969,0.000113635,3.68200e-29,3.13143e-24,CWF19L1,rs11592868


In [69]:
## Estimate rb between our eQTL and GTEx Brain tissues

rb_est <- function(eqtl_ns, eqtl_gtex, ref_pairs) {
    null_ns <- eqtl_ns[gene %in% ref_pairs$gene & `p-value`>0.01,1:3]
    null_gtex <- eqtl_gtex[gene_id %in% ref_pairs$gene_id & `pval_nominal`>0.01,
                           c("slope","gene_name","rsid")]
    null.sumstats <- merge(null_ns, null_gtex, by.x=c("SNP","gene"), 
                           by.y=c("rsid","gene_name"), all=F)
    dtCor <- null.sumstats[, .(mCor = cor(beta,slope)), by=gene]
    re <- mean(dtCor$mCor, na.rm=T)
    
    pair1 <- merge(ref_pairs[,1:2], eqtl_ns, by=c("gene","SNP"), all=F)
    pair1$se <- pair1$beta / pair1[,"t-stat"]
    pair2 <- merge(ref_pairs[,3:4], eqtl_gtex, by=c("gene_id","variant_id"), all=F)
    #pair2$se <- pair2$beta / pair2[,"t-stat"]

    var.e1 <- mean(pair1$se^2)
    var.e2 <- mean(pair2$slope_se^2)

    var.b1 <- var(pair1$beta)
    var.b2 <- var(pair2$slope)

    pairs <- merge(pair1, pair2, by.x=c("gene","SNP"), 
                   by.y=c("gene_name", "rsid"), all=F)
    covar.bhat <- cov(pairs$beta, pairs$slope)

    rb <- (covar.bhat - re*sqrt(var.e1*var.e2))/sqrt(var.b1-var.e1)/sqrt(var.b2-var.e2)
    rb
}

In [67]:
## Standard error for rb between our eqtl and GTEx
rb_se <- function(eqtl_ns, eqtl_gtex, ref_pairs) {
    null_ns <- eqtl_ns[gene %in% ref_pairs$gene & `p-value`>0.01,1:3]
    null_gtex <- eqtl_gtex[gene_id %in% ref_pairs$gene_id & `pval_nominal`>0.01,
                           c("slope","gene_name","rsid")]
    null.sumstats <- merge(null_ns, null_gtex, by.x=c("SNP","gene"), 
                           by.y=c("rsid","gene_name"), all=F)
    dtCor <- null.sumstats[, .(mCor = cor(beta,slope)), by=gene]
    
    pair1 <- merge(ref_pairs[,1:2], eqtl_ns, by=c("gene","SNP"), all=F)
    pair1$se <- pair1$beta / pair1[,"t-stat"]
    pair2 <- merge(ref_pairs[,3:4], eqtl_gtex, by=c("gene_id","variant_id"), all=F)
    
    rbs <- list()
    for(g in ref_pairs$gene) { #Jack-knife approximation
        re <- mean(dtCor$mCor[dtCor$gene!=g], na.rm=T)
    
        var.e1 <- mean(pair1$se[pair1$gene!=g]^2)
        var.e2 <- mean(pair2$slope_se[pair2$gene_name!=g]^2)

        var.b1 <- var(pair1$beta[pair1$gene!=g])
        var.b2 <- var(pair2$slope[pair2$gene_name!=g])

        pairs <- merge(pair1[pair1$gene!=g], pair2[pair2$gene_name!=g], 
                       by.x=c("gene","SNP"), by.y=c("gene_name", "rsid"), all=F)
        covar.bhat <- cov(pairs$beta, pairs$slope)
        #covar.bhat <- cov(pair1$beta[pair1$gene!=g], pair2$beta[pair2$gene!=g])

        rbs[[g]] <- (covar.bhat - re*sqrt(var.e1*var.e2))/sqrt(var.b1-var.e1)/sqrt(var.b2-var.e2)
    }
    rb1 <- unlist(rbs)
    sqrt((length(rb1)-1)/length(rb1)*sum((rb1-mean(rb1))^2)) # Standard error of rb
}

In [61]:
rb_est(eqtls[[1]], cere2, ref_pairs)

Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in c

[1] 0.8403433

In [56]:
rb_se(eqtls[[1]], cere2, ref_pairs)

Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in c

[1] 0.02234221

In [66]:
gtex.store <- "/scratch/midway3/lifanl/"
gtexfs <- list.files(gtex.store,pattern="Brain")
cere <- fread(paste0(gtex.store,"Brain_Cerebellum.allpairs.txt.gz"))
head(cere)

gene_id,variant_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se
<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000227232.5,chr1_13550_G_A_b38,-16003,5,5,0.01196170,0.451465,0.2635580,0.349211
ENSG00000227232.5,chr1_14671_G_C_b38,-14882,2,2,0.00478469,0.532213,0.3441160,0.549778
ENSG00000227232.5,chr1_14677_G_A_b38,-14876,23,23,0.05502390,0.248052,-0.2099730,0.181152
ENSG00000227232.5,chr1_16841_G_T_b38,-12712,19,19,0.04545450,0.885775,0.0272581,0.189464
ENSG00000227232.5,chr1_16856_A_G_b38,-12697,3,3,0.00717703,0.842802,-0.0890186,0.448194
ENSG00000227232.5,chr1_17005_A_G_b38,-12548,6,6,0.01435410,0.123219,-0.5002110,0.322897


In [76]:
### Map to RSID and gene name

#cerebellum <- fread("../../GTEx_Analysis_v8_eQTL/Brain_Cerebellum.v8.signif_variant_gene_pairs.txt.gz")
geneinfo <- fread("../../GTEx_Analysis_v8_eQTL/Brain_Cerebellum.v8.egenes.txt.gz")
cere1 <- merge(cere, geneinfo[,1:2], by="gene_id", all.x=T)

cere2 <- merge(cere1, snpinfo[,c("rsid","variant_id")], by="variant_id", all=F)

head(cere2)

variant_id,gene_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,gene_name,rsid
<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
chr10_100000235_C_T_b38,ENSG00000014919.12,268135,102,120,0.287081,0.6124920,-0.0341321,0.0672604,COX15,rs11596870
chr10_100000235_C_T_b38,ENSG00000023839.10,217503,102,120,0.287081,0.0879439,-0.1264550,0.0736800,ABCC2,rs11596870
chr10_100000235_C_T_b38,ENSG00000055950.16,-987131,102,120,0.287081,0.6893440,-0.0207521,0.0518243,MRPL43,rs11596870
chr10_100000235_C_T_b38,ENSG00000075290.7,-462806,102,120,0.287081,0.8605700,-0.0183043,0.1040510,WNT8B,rs11596870
chr10_100000235_C_T_b38,ENSG00000075826.16,-519629,102,120,0.287081,0.4979820,0.0416309,0.0613000,SEC31B,rs11596870
chr10_100000235_C_T_b38,ENSG00000075891.21,-735368,102,120,0.287081,0.3637060,0.0519238,0.0570100,PAX2,rs11596870


In [81]:
substr(gtexfs,7,nchar(gtexfs)-16)

[1] "Amygdala"                        "Anterior_cingulate_cortex_BA24" 
 [3] "Caudate_basal_ganglia"           "Cerebellar_Hemisphere"          
 [5] "Cerebellum"                      "Cortex"                         
 [7] "Frontal_Cortex_BA9"              "Hippocampus"                    
 [9] "Hypothalamus"                    "Nucleus_accumbens_basal_ganglia"
[11] "Putamen_basal_ganglia"           "Spinal_cord_cervical_c-1"       
[13] "Substantia_nigra"

In [ ]:
res <- matrix(nrow=length(eqtls),ncol=length(gtexfs))
rownames(res) <- names(eqtls)
colnames(res) <- substr(gtexfs,7,nchar(gtexfs)-16)
res.se <- res
for(f in gtexfs) {
    n <- substr(f,7,nchar(f)-16)
    temp <- fread(paste0(gtex.store,f))
    geneinfo <- fread(paste0("../../GTEx_Analysis_v8_eQTL/Brain_",n,".v8.egenes.txt.gz"))
    temp1 <- merge(temp, geneinfo[,1:2], by="gene_id", all.x=T)
    temp2 <- merge(temp1, snpinfo[,c("rsid","variant_id")], by="variant_id", all=F)
    for(e in names(eqtls)) {
        res[e, n] <- rb_est(eqtls[[e]], temp2, ref_pairs)
        res.se[e, n] <- rb_se(eqtls[[e]], temp2, ref_pairs)
    }
}
res

Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”
Warning message in cor(beta, slope):
“the standard deviation is zero”


In [78]:
c(rb_est(eqtls[[1]], cere2, ref_pairs),rb_se(eqtls[[1]], cere2, ref_pairs))

[1] 0.64996557 0.01561291

In [1]:
res

ERROR: Error in eval(expr, envir, enclos): object 'res' not found
